# Langgraph Prebuilt

This noteboook contains the code to run langgraph agent with prebuilt architectures, we look into the React Agents


In [2]:
%load_ext dotenv
%dotenv

In [ ]:
from langgraph.prebuilt import create_react_agent
from pydantic import BaseModel


class WeatherResponse(BaseModel):
    """The response from the weather agent."""

    conditions: str


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"


# Create the agent using the reasoning action agent design
agent = create_react_agent(
    model="openai:gpt-4o",
    tools=[get_weather],
    response_format=WeatherResponse,
    prompt="You are a helpful assistant",
)

# Run the agent
response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)
response["structured_response"]

WeatherResponse(conditions="San Francisco weather is known for its variability throughout the year. Here's a general idea of what you can expect:\n\n### Current Weather\n- **Temperature:** Cool to mild\n- **Wind:** Breezy, especially near the coast\n- **Humidity:** Moderate\n\n### Typical Forecast:\n- **Morning:** Often starts with coastal fog and overcast skies.\n- **Afternoon:** Sunny spells after the fog dissipates, with temperatures typically ranging between 58°F to 67°F.\n- **Evening:** Chilly as the fog rolls back in, wind increases, temperatures drop.\n\n### Notable Features:\n- **Microclimates:** Vary significantly, with cooler temperatures and fog commonly seen in western parts, while eastern areas may experience warmer conditions.\n- **Breezes from the Pacific Ocean** keep the temperatures relatively mild.\n\n### Tips for Visitors:\n- **Dress in Layers:** Be prepared for a range of conditions.\n- **Don't Forget:** A light jacket or sweater, even in summer!\n\nFor today's spec